In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy.stats as stats

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format

# Create locators for ticks on the time axis
years = mdates.YearLocator()
months = mdates.MonthLocator()
years_fmt = mdates.DateFormatter('%Y')

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

### Read the Data

In [ ]:
df_yearly = pd.read_csv('annual_deaths_by_clinic.csv')
# parse_dates avoids DateTime conversion later
df_monthly = pd.read_csv('monthly_deaths.csv', 
                      parse_dates=['date'])

# Preliminary Data Exploration

**Challenge**: Check out these two DataFrames ☝️. 
* What is the shape of df_yearly and df_monthly? How many rows and columns?
* What are the column names?
* Which years are included in the dataset?
* Are there any NaN values or duplicates?
* What were the average number of births that took place per month?
* What were the average number of deaths that took place per month?

In [ ]:
print(df_yearly.shape)
print(df_monthly.shape)

print(df_yearly['year'].unique(), '\n')

In [ ]:
df_yearly.head()

In [ ]:
df_monthly.head()

In [ ]:
print(df_yearly.dtypes)
print(df_monthly.dtypes)

### Check for Nan Values and Duplicates

In [ ]:
print(df_yearly.isna().sum(), '\n')
print(df_monthly.isna().sum())

print(df_yearly.duplicated().sum(), '\n')
print(df_monthly.duplicated().sum())

### Descriptive Statistics

In [ ]:
df_yearly.describe()

In [ ]:
df_monthly.describe()

### Percentage of Women Dying in Childbirth

**Challenge**: How dangerous was childbirth in the 1840s in Vienna? 

* Using the annual data, calculate the percentage of women giving birth who died throughout the 1840s at the hospital.


In comparison, the United States recorded 18.5 maternal deaths per 100,000 or 0.018% in 2013 [(source).](
https://en.wikipedia.org/wiki/Maternal_death#:~:text=The%20US%20has%20the%20%22highest,17.8%20per%20100%2C000%20in%202009)

In [ ]:
df_yearly['pct_death'] = df_yearly['deaths'] / df_yearly['births']
df_yearly.describe()

# Visualise the Total Number of Births 🤱 and Deaths 💀 over Time

### Plot the Monthly Data on Twin Axes

**Challenge**: Create a [Matplotlib chart](https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.pyplot.plot.html) with twin y-axes. It should look something like this:


* Format the x-axis using locators for the years and months (Hint: we did this in the Google Trends notebook)
* Set the range on the x-axis so that the chart lines touch the y-axes
* Add gridlines
* Use `skyblue` and `crimson` for the line colours
* Use a dashed line style for the number of deaths
* Change the line thickness to 3 and 2 for the births and deaths respectively. 
* Do you notice anything in the late 1840s?

In [ ]:
plt.figure(figsize=(14, 8), dpi=200)
plt.title('Total Number of Monthly Births & Deaths', fontsize=18)

ax1 = plt.gca() # get current axis
ax2 = ax1.twinx() # both ax share same x-axis

ax1.grid(color='grey', linestyle='--')

ax1.set_ylabel('Births', color='green', fontsize=18) 
ax2.set_ylabel('Deaths', color='crimson', fontsize=18)

ax1.xaxis.set_major_locator(years)
ax1.xaxis.set_major_formatter(years_fmt)
ax1.xaxis.set_minor_locator(months)

ax1.tick_params(axis='x', labelsize=14, size=14, rotation=45)
ax1.tick_params(axis='y', labelsize=14, size=14)
ax1.tick_params(which='minor', size=6)

ax2.tick_params(axis='both', labelsize=14, size=14)


ax1.plot(
    df_monthly['date'], # x
    df_monthly['births'], # y
    color='green',
    linewidth=2
)

ax2.plot(
    df_monthly['date'], # x
    df_monthly['deaths'], # y
    color='crimson',
    linewidth=2,
    linestyle='--'
)

plt.show()


# The Yearly Data Split by Clinic

Now let's look at the annual data instead. 

**Challenge**: Use plotly to create line charts of the births and deaths of the two different clinics at the Vienna General Hospital. 
* Which clinic is bigger or more busy judging by the number of births?
* Has the hospital had more patients over time? 
* What was the highest number of deaths recorded in clinic 1 and clinic 2?

In [ ]:
line = px.line(
    data_frame=df_yearly,
    x='year',
    y='births',
    color='clinic',
    title='Total Yearly Births By Clinic'
)

line.show()

In [ ]:
line = px.line(
    data_frame=df_yearly,
    x='year',
    y='deaths',
    color='clinic',
    title='Total Yearly Deaths By Clinic'
)

line.show()

### Calculate the Proportion of Deaths at Each Clinic

**Challenge**: Calculate the proportion of maternal deaths per clinic. That way we can compare like with like. 
* Work out the percentage of deaths for each row in the `df_yearly` DataFrame by adding a column called "pct_deaths". 
* Calculate the average maternal death rate for clinic 1 and clinic 2 (i.e., the total number of deaths per the total number of births).
* Create another plotly line chart to see how the percentage varies year over year with the two different clinics.
* Which clinic has a higher proportion of deaths?
* What is the highest monthly death rate in clinic 1 compared to clinic 2?

In [ ]:
df_yearly.groupby('clinic').agg({'pct_death': pd.Series.mean})

### Plotting the Proportion of Yearly Deaths by Clinic

In [ ]:
line = px.line(
    data_frame=df_yearly,
    x='year',
    y='pct_death',
    color='clinic',
    title='Total Proportion Deaths By Clinic'
)

line.show()

# The Effect of Handwashing

Dr Semmelweis made handwashing obligatory in the summer of 1947. In fact, he ordered people to wash their hands with clorine (instead of water).

In [ ]:
# Date when handwashing was made mandatory
handwashing_start = pd.to_datetime('1847-06-01')

**Challenge**: 
* Add a column called "pct_deaths" to `df_monthly` that has the percentage of deaths per birth for each row. 
* Create two subsets from the `df_monthly` data: before and after Dr Semmelweis ordered washing hand.
* Calculate the average death rate prior to June 1947.
* Calculate the average death rate after June 1947.

In [ ]:
df_monthly['pct_deaths'] = df_monthly['deaths'] / df_monthly['births']

before_hand_wash = df_monthly[df_monthly['date'] < '1847-06-01']
after_hand_wash = df_monthly[df_monthly['date'] >= '1847-06-01']

In [ ]:
print(f'Before Handwash Death: {before_hand_wash["pct_deaths"].mean()}')
print(f'After handwash Death: {after_hand_wash["pct_deaths"].mean()}')

### Calculate a Rolling Average of the Death Rate

**Challenge**: Create a DataFrame that has the 6 month rolling average death rate prior to mandatory handwashing.

*Hint*: You'll need to set the dates as the index in order to avoid the date column being dropped during the calculation.

In [ ]:
roll_df = before_hand_wash.set_index('date')
roll_df = roll_df.rolling(window=6).mean()

### Highlighting Subsections of a Line Chart

**Challenge**: Copy-paste and then modify the Matplotlib chart from before to plot the monthly death rates (instead of the total number of births and deaths). The chart should look something like this:

<img src=https://i.imgur.com/X6TQe0R.png width=500>

* Add 3 seperate lines to the plot: the death rate before handwashing, after handwashing, and the 6-month moving average before handwashing.
* Show the monthly death rate before handwashing as a thin dashed black line. 
* Show the moving average as a thicker, crimon line.
* Show the rate after handwashing as a skyblue line with round markers. 
* Look at the [code snippet in the documentation to see how you can add a legend](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.legend.html) to the chart.

In [ ]:
plt.figure(figsize=(14, 8), dpi=200)
plt.title('Percentage of Monthly Deaths over Time', fontsize=18)
plt.ylabel('Percentage of Deaths', color='crimson', fontsize=18)

ax = plt.gca()

ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
ax.set_xlim([df_monthly.date.min(), df_monthly.date.max()])

ax.tick_params(axis='x', labelsize=14, size=14, rotation=45)
ax.tick_params(which='minor', size=6)

plt.grid(color='grey', linestyle='--')

bw_line = plt.plot(
    before_hand_wash['date'], # x
    before_hand_wash['pct_deaths'],
    color='black',
    linewidth=1,
    linestyle='-',
    label='Before Handwashing'
)

aw_line = plt.plot(
    after_hand_wash['date'],
    after_hand_wash['pct_deaths'],
    color='blue',
    linewidth=1,
    label='After Handwashing'
)

mvg_avg = plt.plot(
    roll_df.index,
    roll_df['pct_deaths'],
    color='red',
    linewidth=2,
    linestyle='dotted',
    label='6mth rolling average'
)

plt.legend(),
plt.show()

### Statistics - Calculate the Difference in the Average Monthly Death Rate

**Challenge**:
* What was the average percentage of monthly deaths before handwashing? 
* What was the average percentage of monthly deaths after handwashing was made obligatory?
* By how much did handwashing reduce the average chance of dying in childbirth in percentage terms?
* How do these numbers compare to the average for all the 1840s that we calculated earlier? 
* How many times lower are the chances of dying after handwashing compared to before?

In [ ]:
avg_prob_before = before_hand_wash.pct_deaths.mean() * 100
print(f'Chance of death during childbirth before handwashing: {avg_prob_before:.3}%.')

avg_prob_after = after_hand_wash.pct_deaths.mean() * 100
print(f'Chance of death during childbirth AFTER handwashing: {avg_prob_after:.3}%.')

mean_diff = avg_prob_before - avg_prob_after
print(f'Handwashing reduced the monthly proportion of deaths by {mean_diff:.3}%!')

times = avg_prob_before / avg_prob_after
print(f'This is a {times:.2}x improvement!')

### Use Box Plots to Show How the Death Rate Changed Before and After Handwashing

**Challenge**: 
* Use [NumPy's `.where()` function](https://numpy.org/doc/stable/reference/generated/numpy.where.html) to add a column to `df_monthly` that shows if a particular date was before or after the start of handwashing. 
* Then use plotly to create box plot of the data before and after handwashing. 
* How did key statistics like the mean, max, min, 1st and 3rd quartile changed as a result of the new policy?



In [ ]:
df_monthly['washing_hands'] = np.where(
    df_monthly['date'] < '1847-06-01','No', 'Yes'
) # 'No' if true, 'Yes' if false

In [ ]:
box = px.box(
                data_frame=df_monthly, 
                x='washing_hands', 
                y='pct_deaths',
                color='washing_hands',
                title='How Have the Stats Changed with Handwashing?')
    
box.update_layout(xaxis_title='Washing Hands?',
                    yaxis_title='Percentage of Monthly Deaths',)
    
box.show()

### Use Histograms to Visualise the Monthly Distribution of Outcomes

**Challenge**: Create a [plotly histogram](https://plotly.com/python/histograms/) to show the monthly percentage of deaths. 

* Use docs to check out the available parameters. Use the [`color` parameter](https://plotly.github.io/plotly.py-docs/generated/plotly.express.histogram.html) to display two overlapping histograms.
* The time period of handwashing is shorter than not handwashing. Change `histnorm` to `percent` to make the time periods comparable. 
* Make the histograms slighlty transparent
* Experiment with the number of bins on the histogram. Which number work well in communicating the range of outcomes?
* Just for fun, display your box plot on the top of the histogram using the `marginal` parameter. 

In [ ]:
hist = px.histogram(
    data_frame=df_monthly,
    x='pct_deaths',
    color='washing_hands',
    nbins=30,
    opacity=0.5,
    barmode='overlay',
    histnorm='percent',
    marginal='box'
)

hist.update_layout(
    xaxis_title='Porportion of Monthly Deaths',
    yaxis_title='Count'
)

hist.show()

### Use a Kernel Density Estimate (KDE) to visualise a smooth distribution

**Challenge**: Use [Seaborn's `.kdeplot()`](https://seaborn.pydata.org/generated/seaborn.kdeplot.html) to create two kernel density estimates of the `pct_deaths`, one for before handwashing and one for after. 

* Use the `shade` parameter to give your two distributions different colours. 
* What weakness in the chart do you see when you just use the default parameters?
* Use the `clip` parameter to address the problem. 


In [ ]:
plt.figure(dpi=200)

sb.kdeplot(
    before_hand_wash['pct_deaths'], shade=True, clip=(0, 1), label='before handwash'
)

sb.kdeplot(
    after_hand_wash['pct_deaths'], shade=True, clip=(0, 1), label='after'
)

plt.legend()
plt.xlim(0, 0.4)
plt.title('Est. Distribution of Monthly Death Rate Before & After Handwash')
plt.show()

### Use a T-Test to Show Statistical Significance

**Challenge**: Use a t-test to determine if the differences in the means are statistically significant or purely due to chance. 

If the p-value is less than 1% then we can be 99% certain that handwashing has made a difference to the average monthly death rate. 

* Import `stats` from scipy
* Use the [`.ttest_ind()` function](https://docs.scipy.org/]doc/scipy/reference/generated/scipy.stats.ttest_ind.html) to calculate the t-statistic and the p-value
* Is the difference in the average proportion of monthly deaths statistically significant at the 99% level? 



In [ ]:
t_stat, p_value = stats.ttest_ind(a=before_hand_wash.pct_deaths, 
                                    b=after_hand_wash.pct_deaths)
print(f'p-palue is {p_value:.10f}')
print(f't-statstic is {t_stat:.4}')